# Grounding Guardrail 

### Fact-Checking

The goal of the self-check fact-checking output rail is to ensure that the answer to a RAG (Retrieval Augmented Generation) query is grounded in the provided evidence extracted from the knowledge base (KB).

NeMo Guardrails uses the concept of **relevant chunks** (which are stored in the `$relevant_chunks` context variable) as the evidence against which fact-checking should be performed. The relevant chunks can be extracted automatically, if the built-in knowledge base support is used, or provided directly alongside the query (see the [Getting Started Guide example](../getting_started/7_rag)).

**IMPORTANT**: The performance of this rail is strongly dependent on the capability of the LLM to follow the instructions in the `self_check_facts` prompt.

In [35]:
import sys
import os

module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [36]:
import json
import boto3
from utils import bedrock, print_ww
import pandas as pd

# This helper function encompasses the process of initializing NeMo Guardrails and generating Rails based on a specified configuration.
from nemoguardrails import LLMRails, RailsConfig

# BedrockModels is a "Singleton" class which initializes the necessary models for the notebook.
from models import BedrockModels

# This creates a pywidget chat window for testing
from models import ChatComponent

## Initalize bedrock client

In [37]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Define function to create a LLMRails instance with Claudev2 as LLM and a certain rail configuration

In [38]:
# This helper function encapsulates the necessary steps to bootstrap
# NeMo Guardrails and returns Rails based on a given configuration.
def bootstrap_bedrock_nemo_guardrails(rail_config_path: str) -> LLMRails:

    #1. initialize rails config
    config = RailsConfig.from_path(f"NeMo/rails/{rail_config_path}/config")

    # initialize bedrock models
    # you can pass model id as string or use the default model id 'anthropic.claude-v2'
    bedrock_models = BedrockModels
    bedrock_models.init_bedrock_client(boto3_bedrock)
    bedrock_models.init_llm('anthropic.claude-v2')

    # 2. bootstraps NeMo Guardrails with the necessary resources
    app = LLMRails(config=config, llm=bedrock_models.llm,  verbose=True)
    
    return app


In [39]:
rails = bootstrap_bedrock_nemo_guardrails("grounding")

Entered verbose mode.


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

In [40]:
import nest_asyncio
nest_asyncio.apply()

### Enable or disable the _check_facts_ action by setting the boolean variable below

In [41]:
response = await rails.generate_async(messages=[
    {"role": "context", "content": {"check_facts": False}},
    {
    "role": "user",
    "content": "How many vacation days do I have? as a regular employee"}])

print_ww(response["content"])

Event UtteranceUserActionFinished {'final_transcript': 'How many vacation days do I have? as a regular employee'}
Event StartInternalSystemAction {'uid': 'a67a478e-2485-4a99-9018-f9b71e2b188e', 'event_created_at': '2024-02-19T17:27:29.706213+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'create_event', 'action_params': {'event': {'_type': 'UserMessage', 'text': '$user_message'}}, 'action_result_key': None, 'action_uid': '46a9e80e-41be-4ddf-80cb-837427ac38a6', 'is_system_action': True}
Executing action create_event
Event UserMessage {'uid': '8ec919bf-2874-4a78-96f7-4a734efe7026', 'event_created_at': '2024-02-19T17:27:29.706694+00:00', 'source_uid': 'NeMoGuardrails', 'text': 'How many vacation days do I have? as a regular employee'}
Event StartInternalSystemAction {'uid': '338dc048-4ac6-4696-b6de-16933753cd37', 'event_created_at': '2024-02-19T17:27:29.707766+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'generate_user_intent', 'action_params': {}, 'action_result_key': N

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Event InternalSystemActionFinished {'uid': '731e2b6b-0810-4369-a063-42d872e0b145', 'event_created_at': '2024-02-19T17:27:38.426247+00:00', 'source_uid': 'NeMoGuardrails', 'action_uid': '4ea44c31-fcd8-410e-9e83-c02c877f2ece', 'action_name': 'retrieve_relevant_chunks', 'action_params': {}, 'action_result_key': None, 'status': 'success', 'is_success': True, 'return_value': 'Employees must provide reasonable notice for time off, whenever possible. Unused vacation and sick leave will be paid out upon termination.\n\nEmployees are expected to maintain a healthy work-life balance and are encouraged to use their time off when needed.\n* Reviews will be conducted semi-annually, in January and July.\n* Reviews will be based on performance against expectations, goals, and contributions to the company.\n* Employees will receive feedback on their strengths, areas for improvement, and development opportunities.\n* Employees will have the opportunity to provide feedback on their manager and the compa

In [42]:
for call in range(len(rails.explain().llm_calls)):
    call_metadata = rails.explain().llm_calls[call]
    
    print_ww(f"task: {call+1}", call_metadata.task)
    print_ww(f"prompt: {call+1}", call_metadata.prompt)
    print_ww(f"completion: {call+1}", call_metadata.completion)

    print("------\n")

task: 1 generate_user_intent
prompt: 1 System: """
Below is a conversation between a user and a bot called the ABC Bot.
The bot is designed to answer employee questions about the ABC Company.
The bot is knowledgeable about the employee handbook and company policies.
If the bot does not know the answer to a question, it truthfully says it does not know.

Your task is to generate a short summary called user intent for the last user message in a
conversation.
"""

# This is how a conversation between a user and the bot can go:
human "Hi there. Can you help me with some questions I have about the company?"
express greeting and ask for assistance
assistant express greeting and confirm and offer assistance
"Hi there! I'm here to help answer any questions you may have about the ABC Company. What would you
like to know?"
human "What are the companies employee benefits?"
ask question about benefits
assistant respond to question about benefits
"The ABC Company provides comprehensive employee ben

# Try-Out

## Conclusion and Wrap Up

Congratulations on completing this module on establishing guardrails on chatbots and large language models! Guardrails for LLMs act as control mechanisms to ensure that LLM-generated responses remain within desired parameters, preventing and correcting unwanted content output. They are programmable to follow specified interaction paths, respond to certain user requests in particular ways, and maintain a designated language style, among other controls.   

Designing guardrails before deploying chatbots and large language models to production should be a top priority during the early stages. Understanding how these guardrails function is crucial for enhancing your solutions and effectively positioning and configuring them. It's advisable to adopt and implement guardrails from the outset, ensuring the creation of secure and well-moderated language generation systems. Great job!

#### In this demo, we used Amazon Bedrock, NeMo Guardrails, and Faiss to set guardrails on conversational bots and showcased the following applications:

- **Jailbreaking Rail:** Restricts AI from deviating from a set response format.
- **Topical Rail:** Ensures that chatbots stay on topic.
- **Moderation Rail:** Moderates AI responses to maintain a neutral stance.

### Further Reading & Experimentation

- Experiment with:
  - Different Vector Stores
  - Knowledge Bases
  - Guardrail Strategies
  - Prompts and Instructions
  - Leverage various models available under Amazon Bedrock to see alternate outputs

- Reading
  - Amazon Bedrock 
  - Amazon Vector Stores 
    - Amazon OpenSearch 
    - Amazon RDS and PGVector
    - Amazon Kendra 
  - NeMo Guardrails 
  - Faiss 


# Thank You
